In [ ]:
pip install torch torchvision matplotlib scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from torchvision import datasets, transforms, models
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report
from PIL import Image


In [ ]:
REAL_PATH = "/content/drive/MyDrive/Projects/real_1"
FORGED_PATH = "/content/drive/MyDrive/Projects/forged_1"

In [ ]:
root_dir = "/content/signatures_combined"
real_out = os.path.join(root_dir, "real")
forged_out = os.path.join(root_dir, "forged")

os.makedirs(real_out, exist_ok=True)
os.makedirs(forged_out, exist_ok=True)

In [ ]:
# Copy files from Drive to Colab workspace
import shutil
for file in os.listdir(REAL_PATH):
    shutil.copy(os.path.join(REAL_PATH, file), os.path.join(real_out, file))
for file in os.listdir(FORGED_PATH):
    shutil.copy(os.path.join(FORGED_PATH, file), os.path.join(forged_out, file))


In [ ]:
# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
# Load dataset
dataset = datasets.ImageFolder(root=root_dir, transform=transform)

In [ ]:
# Split dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

In [ ]:
# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 129MB/s]


In [ ]:
# Freeze feature extractor
for param in model.parameters():
    param.requires_grad = False


In [ ]:
# Replace final classification layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
# Training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/10, Loss: 0.4143
Epoch 2/10, Loss: 0.2335
Epoch 3/10, Loss: 0.2610
Epoch 4/10, Loss: 0.2158
Epoch 5/10, Loss: 0.1879
Epoch 6/10, Loss: 0.1835
Epoch 7/10, Loss: 0.1641
Epoch 8/10, Loss: 0.1594
Epoch 9/10, Loss: 0.1837
Epoch 10/10, Loss: 0.1661


In [ ]:
# Evaluation
model.eval()
all_preds = []
all_labels = []

In [ ]:
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=dataset.classes))

              precision    recall  f1-score   support

      forged       0.94      0.99      0.96       146
        real       0.99      0.95      0.97       184

    accuracy                           0.97       330
   macro avg       0.96      0.97      0.97       330
weighted avg       0.97      0.97      0.97       330



In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Projects/signature_model_10.pth')


In [ ]:
# Prediction function
def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)

    label = dataset.classes[pred.item()]
    return label

# Run prediction
test_image_pathr="/content/drive/MyDrive/Projects/real_1/01_018.png"
test_image_pathf="/content/drive/MyDrive/Projects/forged_1/01_0105041.PNG"
result = predict_image(test_image_pathr)
print(f"Prediction: {result.upper()}")


Prediction: REAL
